# 用 Pandas 產生樞紐報表 (pivot table)

樞紐報表是 Excel 非常强大的功能，可以將大量的資料，依照制定的資料樣式重新分類整合。只要確定了新分類的表格樣式，透過簡單的操作就可以輕鬆產生新的報表。

![](https://drive.google.com/uc?export=download&id=1a55oDUBjn1Cs5Xyqzy0FulXP8hjsSYXE)

若今天樞紐報表十分複雜，在圖像化界面上就很難快速的透過手動的方式快速的產生樞紐報表。

但是今天我們若希望透過程式化的方式來產生樞紐報表，我們其實可以活用 Pandas 的 DataFrame 產生樞紐報表，不但效率快，而且語法十分簡單。

In [ ]:
import xlwings as xw
# 請輸入 pivot_table.xlsx 的絕對路徑
wb = xw.Book(r"pivot_table.xlsx")

data_sheet = wb.sheets["銷售數據"]

report_sheet = wb.sheets["報告"]

## 如何能夠把工作表内的資料放入 DataFrame?

若我們今天直接執行 **range(起點:結束點).value**，預設上 **xlwings** 會將資料以二維串列(2d list)的形式將資料回傳給我們

但是，若今天我們希望能夠該範圍的資料放入一個 DataFrame 呢？

```python
data_sheet.range("A1:B2").value

# [
#    ['訂單號碼', '日期'],
#    ['A0001', datetime.datetime(2017, 1, 1, 0, 0)]
# ]
```

##  range().options(format, expand=)

`range()` 的 `options()` 方法會要求使用者輸入兩個參數： **format** 以及 **expand** 

```python
range(範圍).options(format, expand="方向")
```


`format` 是你希望 xlwings 將**一個範圍的值以什麽樣的資料結構回傳**

`expand` 是讓 xlwings 自動偵測試算表資料的範圍

舉例來説，設定 `down` 代表往下搜尋，而設定 `table` 則是將整個試算表，連續的資料一次性的搜出並存入指定的資料結構内。

請嘗試執行以下程式碼：

```python
import numpy as np
# 將工作表的資料封裝成 Numpy Array
data_sheet.range("A1").options(np.array, expand="table").value
```


## options 範例

```python
import pandas as pd
# 這行代表以 A1 為起點，同時往下與往右搜尋，找出了連續範圍的右下角，將銷售資料截取出來，存入 DataFrame
df = data_sheet.range("A1").options(pd.DataFrame, expand="table").value

```
所以從圖示來看，`expand="table"` 就像是：
![](https://drive.google.com/uc?export=download&id=1AXJ4oKcqCFu4XyT9VRdb-FBrxvXgYNGm)


In [ ]:
import pandas as pd
# 將銷售資料截取出來，存入 DataFrame
df = data_sheet.range("A1").options(pd.DataFrame, expand="table").value
df

In [ ]:
# 將 DataFrame 的索引設定成日期
df.set_index("日期")

## Pandas 的 groupby 功能

先厘清我們是要以哪一欄做分類，將該欄的名稱以字串的方式指定給 **by**

**DataFrame.groupby(by="欄位名稱")**

In [ ]:
df.groupby(by="產品", sort=True).sum()

In [ ]:
# 以產品的項目對 DataFrame 的資料做分類，將分類出來的結果做加總
report_df = df.groupby(by="產品").sum()
report_df

In [ ]:
# 問題是加總起來的資料並非依照大小來排列，所以我們就手動來排列資料
final_report_df = report_df.sort_values(by="金額", ascending=False)
final_report_df

## df.sum()

`df["欄名"].sum()` 會將指定一欄的資料加總起來 

In [ ]:
final_report_df['金額'] / final_report_df['金額'].sum()

In [ ]:
# 將所有的金額都除上加總，算出每一種水果的銷量的百分比
final_report_df['金額'] / final_report_df['金額'].sum()

In [ ]:
# 問題是這樣的數字依然不明顯，所以我們另外在 DataFrame 加入新的一欄，計算出每一種水果的銷量的百分比
report_df["比例%"] =  report_df["金額"] * 100 / report_df["金額"].sum()
report_df

In [ ]:
# 問題是加總起來的資料並非依照大小來排列，所以我們就手動來排列資料
final_report_df = report_df.sort_values(by="金額", ascending=False)
final_report_df

In [ ]:
# 最後將這個 DataFrame 的資料寫回到 Excel
# 簡單來説，就是設定 DataFrame 資料在 Excel 的左上角
report_sheet = wb.sheets["報告"]
report_sheet.range('A1').value = final_report_df

![](https://drive.google.com/uc?export=download&id=16Wq9tV1QRItjlmjhlyE6gdXWPxrkMa4D)

# 令一種實作樞紐報表的功能：pivot_table 方法

用法：
```python
pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
```

看似很複雜，但從實務的角度來看，需要搞懂的是以下幾個最重要的參數：

- index
- columns
- aggfunc

官方文件：[連結](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [ ]:
# 首先，任何樞紐報表都需要 index，也就是分類用的欄位，下面這邊我們是以 “產品” 這一欄爲例
pd.pivot_table(df, index=["產品"])

In [ ]:
# 接下來我們需要指定一個聚合資料時的函數 / 算法，這邊我們會使用 numpy 的加總函數 (np.sum)
# pivot_table 預設的 aggfunc 是 mean，代表計算平均值
import numpy as np

pd.pivot_table(df, index=["產品"], aggfunc=np.sum)

In [ ]:
# 針對加總之後的金額做排序
report = pd.pivot_table(df, index=["產品"], aggfunc=np.sum).sort_values(by="金額", ascending=False)
report

In [ ]:
report_sheet.range("A1").value = report

In [ ]:
pd.pivot_table(df, index="產品", columns="業務員", aggfunc=np.sum)

In [ ]:
# 也可從時間進行分層
pd.pivot_table(df, index=["產品"], columns=["日期"])

## 練習：

請利用樞紐報表計算出每一個業務員的業績，並且以大到小進行排序

# pivot_table vs groupby

兩者都是產生樞紐報表的功能，而 pivot_table 由於有 columns 參數，所以比 groupby **稍微更彈性，但是使用上也更複雜一點。**

In [ ]:
import pandas as pd
# 將工作表的資料封裝成 DataFrame
data_sheet.range("A1").options(pd.DataFrame, expand="table").value

# 產生圓餅圖

用 Python 產生樞紐報表的圓餅圖可以使用兩種不同的方式：

- 使用 Excel 原廠 Chart 物件
- 使用 Matplotlib

我們先來看用 Python 操作 Excel 原廠的圖表物件

In [ ]:
chart = report_sheet.charts.add()
# 使用 expand 將試算表内所有、連續的資料撈出
chart.set_source_data(report_sheet.range('A1').expand())
chart.chart_type = 'pie'
# 圓餅圖最上方等於 E1 儲存格的上邊
chart.top = report_sheet.range('E1').top
# 圓餅圖最左方等於 E1 儲存格的左邊
chart.left = report_sheet.range('E1').left
# 設定圓餅圖最擡頭（Windows 限定）
# chart.api[1].ChartTitle.Text = '產品銷售金額比例'

![](https://drive.google.com/uc?export=download&id=1yf4D-EeGTswqdemupTY2tIRkBk_FfoJz)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 產生資料
labels = final_report_df.index
data = final_report_df["金額"]
 
# 畫出圖表
plt.rcParams['font.sans-serif'] = ['SimHei']
fig = plt.figure()
plt.pie(data, labels=labels, autopct='%1.1f%%', shadow=True)
plt.title("產品銷售比例圖")
# 設定圖例
plt.legend(loc='right')
plt.axis('equal')
plt.show()

## 若遇到中文不能顯示的問題

Matplotlib 解決中文亂碼教學：[連結](https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/359974/)

# 將圖表輸出到 Excel 上

In [ ]:
plot = report_sheet.pictures.add(fig, left=report_sheet.range('E1').left, 
                                 top=report_sheet.range('E1').top)

![](https://drive.google.com/uc?export=download&id=1OVEhF24_2VDpgo0PdqJY6PgX9WJ2YG9n)

# 完整版程式碼

In [ ]:
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt

# 請輸入 pivot_table.xlsx 的絕對路徑
wb = xw.Book(r"pivot_table.xlsx")

data_sheet = wb.sheets["銷售數據"]
report_sheet = wb.sheets["報告"]
# 將銷售資料截取出來，存入 DataFrame
df = data_sheet.range("A1").options(pd.DataFrame, expand="table").value

report_df = df.groupby(by="產品").sum()
final_report_df = report_df.sort_values(by="金額", ascending=False)
# 問題是這樣的數字依然不明顯，所以我們另外在 DataFrame 加入新的一欄，計算出每一種水果的銷量的百分比
report_df["比例%"] =  report_df["金額"] * 100 / report_df["金額"].sum()
final_report_df = report_df.sort_values(by="金額", ascending=False)
report_sheet.range("A1").value = final_report_df
# 產生資料
labels = final_report_df.index
data = final_report_df["金額"]
# 畫出圖表
plt.rcParams['font.sans-serif'] = ['SimHei']
fig = plt.figure()
plt.pie(data, labels=labels, autopct='%1.1f%%', shadow=True)
plt.title("產品銷售比例圖")
# 設定圖例
plt.legend(loc='right')
plt.axis('equal')
plt.show()

plot = report_sheet.pictures.add(fig, left=report_sheet.range('E1').left, 
                                 top=report_sheet.range('E1').top)

## 測試：輸出完整報表

希望能夠產生類似像以下格式的報表：

![](https://drive.google.com/uc?export=download&id=16KE3RTbX02KYh0ATo27yCraqLRv1XGLP)

此時我們可以先將兩個樞紐報表，以及圓餅圖會出現的位置定義名稱：

`pivot_product` 代表產品樞紐報表的名稱

`pivot_salesmen` 代表銷售員樞紐報表的名稱

`pie_chart_product` 代表圓餅圖的名稱

最後就可以將 `DataFrame` 各自寫入不同的範圍：

In [ ]:
test_sheet = wb.sheets["報表2"]

In [ ]:
test_sheet.range("pivot_product").value = final_report_df

In [ ]:
# 產生資料
labels = final_report_df.index
data = final_report_df["金額"]
# 畫出圖表
plt.rcParams['font.sans-serif'] = ['SimHei']
fig = plt.figure()
plt.pie(data, labels=labels, autopct='%1.1f%%', shadow=True)
plt.title("產品銷售比例圖")
# 設定圖例
plt.legend(loc='right')
plt.axis('equal')
plt.show()

plot = test_sheet.pictures.add(fig, left=test_sheet.range('pie_chart_product').left, 
                                 top=test_sheet.range('pie_chart_product').top)

In [ ]:
report2 = df.groupby("業務員").sum().sort_values("金額", ascending=False)
report2["比例%"] = report2["金額"] * 100 / report2["金額"].sum()

test_sheet.range("pivot_salesmen").value = report2